In [1]:
# 1) new atom Encoding
# 2) scaffold splitting
# 3) conformer generation

# frad and conforge at runtime is it feasible?

In [2]:
import numpy as np
import sys
import os

import CDPL.Chem as Chem
import CDPL.ConfGen as ConfGen
import rdkit
import CDPL.Chem as CDPLChem

# import rdkit

smiles=["CC(C)Cl", "CCC(C)CO", "CCCCCCCO"]# , "CCCCCCCC(=O)OC" , "c3ccc2nc1ccccc1cc2c3" , "Nc2cccc3nc1ccccc1cc23" , "C1CCCCCC1" ]

In [3]:
def generateConformationEnsembles(mol: Chem.BasicMolecule, conf_gen: ConfGen.ConformerGenerator):
  """
  Generates a conformation ensemble for the argument molecule using the provided initialized ConfGen.ConformerGenerator instance.

  Parameters:
  - mol (Chem.BasicMolecule): Molecule to generate a conformation ensemble for.
  - conf_gen (ConfGen.ConformerGenerator): Instance of the ConfGen.ConformerGenerator class.

  Returns:
  - int: Status code indicating the success of the conformation ensemble generation.
  - int: Number of generated conformers.
  """
  # prepare the molecule for conformer generation
  ConfGen.prepareForConformerGeneration(mol)

  # generate the conformer ensemble
  status = conf_gen.generate(mol)
  num_confs = conf_gen.getNumConformers()

  # if sucessful, store the generated conformer ensemble as
  # per atom 3D coordinates arrays (= the way conformers are represented in CDPKit)
  if status == ConfGen.ReturnCode.SUCCESS or status == ConfGen.ReturnCode.TOO_MUCH_SYMMETRY:
      conf_gen.setConformers(mol)
  else:
      num_confs = 0

  # return status code and the number of generated conformers
  return (status, num_confs)

In [4]:
def get_conformer_generator():
  # Settings
  max_time = 36000 # Max. allowed molecule processing time in seconds (default: 3600 sec)
  min_rmsd = 0.5 # Output conformer RMSD threshold (default: 0.5)
  e_window = 20.0 # Output conformer energy window (default: 20.0)
  max_confs = 100 # Max. output ensemble size (default: 100)

  # create and initialize an instance of the class ConfGen.ConformerGenerator which
  # will perform the actual conformer ensemble generation work
  conf_gen = ConfGen.ConformerGenerator()
  conf_gen.settings.timeout = max_time * 1000          # apply the -t argument
  conf_gen.settings.minRMSD = min_rmsd                 # apply the -r argument
  conf_gen.settings.energyWindow = e_window            # apply the -e argument
  conf_gen.settings.maxNumOutputConformers = max_confs # apply the -n argument
  return conf_gen

In [ ]:
import numpy as np
from rdkit import Chem as rdChem
from rdkit.Geometry import rdGeometry
from rdkit.Chem import AllChem, rdMolTransforms

def get_dihedral_angles(mol):
  # Find all dihedral angles (torsions)
  dihedral_indices = []

  # from frad repo
  torsionSmarts = '[!$(*#*)&!D1]-&!@[!$(*#*)&!D1]'
  # from chatgpt
  # torsionSmarts = '[!#1]~[!#1]~[!#1]~[!#1]'
  for torsion in mol.GetSubstructMatches(rdChem.MolFromSmarts(torsionSmarts)): dihedral_indices.append(torsion)

  # Calculate all dihedral angles
  dihedral_angles = []
  conf = mol.GetConformer()
  for dihedral in dihedral_indices:
    dihedral_angles.append(rdMolTransforms.GetDihedralDeg(conf, dihedral[0], dihedral[1], dihedral[2], dihedral[3]))
  return np.array(dihedral_angles)

In [ ]:
def generate_conformers(smi, N:int=10):
  '''
  N: num of conformers to select
  '''
  mol = CDPLChem.parseSMILES(smi)
  try:
      # generate conformer ensemble for read molecule
      status, num_confs = generateConformationEnsembles(mol, get_conformer_generator())

      # output generated ensemble (if available)
      if num_confs > 0:
          tmp_name = "./tmp.sdf"
          writer = Chem.MolecularGraphWriter(tmp_name)
          if not writer.write(mol):
              sys.exit('Error: output of conformer ensemble for molecule %s failed' % smi)
  except Exception as e:
      sys.exit('Error: conformer ensemble generation or output for molecule %s failed: %s' % (smi, str(e)))

  writer.close()
  tmp_mol_ensemble = rdkit.Chem.rdmolfiles.SDMolSupplier(tmp_name, removeHs=False)
  tmp_mol_ensemble = list(tmp_mol_ensemble)

  # try to get conf, if error then drop the mol, careful with indices
  mol_ensemble = [m for m in tmp_mol_ensemble if m.GetConformer() !=-1]
  print(len(mol_ensemble))
  if len(mol_ensemble) < N: return mol_ensemble

  dihedral_fingerprints = [get_dihedral_angles(mol) for mol in mol_ensemble]

  # Compute the dot product matrix
  n = len(dihedral_fingerprints)
  dot_product_matrix = np.zeros((n, n))
  for i in range(n):
      for j in range(i + 1, n):
          dot_product_matrix[i, j] = np.dot(dihedral_fingerprints[i], dihedral_fingerprints[j])
          dot_product_matrix[j, i] = dot_product_matrix[i, j]

  sum_of_dot_prods = dot_product_matrix.sum(-1)
  sorted_indices = np.argsort(sum_of_dot_prods)
  # Get the indices of the 5 lowest values
  lowest_indices = sorted_indices[:N]

  if os.path.exists(tmp_name): os.remove(tmp_name)

  return [mol_ensemble[i] for i in lowest_indices]



In [11]:
d = generate_conformers('[H]c1c([H])c([H])c(N(C(=O)C([H])([H])C([H])([H])[H])C2([H])C([H])([H])C([H])([H])N(C([H])([H])C([H])([H])c3c([H])c([H])c([H])c([H])c3[H])C([H])([H])C2([H])[H])c([H])c1[H]')

64
[[     0.          96533.92913327  94089.99929102 ... 266138.41732614
  201565.30113964 216707.9227926 ]
 [ 96533.92913327      0.         376569.67543564 ... 351263.51099943
  286225.24792106 383550.71108261]
 [ 94089.99929102 376569.67543564      0.         ... 207131.43206721
  271689.72831157 405106.95686096]
 ...
 [266138.41732614 351263.51099943 207131.43206721 ...      0.
  272757.80530541 271523.19944007]
 [201565.30113964 286225.24792106 271689.72831157 ... 272757.80530541
       0.         206421.6739375 ]
 [216707.9227926  383550.71108261 405106.95686096 ... 271523.19944007
  206421.6739375       0.        ]]


In [8]:
d

In [9]:
from my_general_utils import visualize_3d_mols
visualize_3d_mols(d)


3Dmol.js failed to load for some reason. Please check your browser console for error messages.